<img src="images/header-transparent.png" alt="Logo UCLM-ESII" align="right">

<br><br><br><br>
<h2><font color="#92002A" size=4>Trabajo Fin de Máster</font></h2>

<h1><font color="#6B001F" size=5>SERENDIPITY: Servicio web para la recomendacIón de playlists a partir de otra playlist</font></h1>
<h2><font color="#92002A" size=3>Parte 5 - Servicios de Machine Learning</font></h2>

<br>
<div style="text-align: right">
    <font color="#B20033" size=3><strong>Autor</strong>: <em>Miguel Ángel Cantero Víllora</em></font><br>
    <br>
    <font color="#B20033" size=3><strong>Directores</strong>: <em>José Antonio Gámez Martín</em></font><br>
    <font color="#B20033" size=3><em>Juan Ángel Aledo Sánchez</em></font><br>
    <br>
<font color="#B20033" size=3>Máster Universitario en Ingeniería Informática</font><br>
<font color="#B20033" size=2>Escuela Superior de Ingeniería Informática | Universidad de Castilla-La Mancha</font>

</div>

---

<br>


<a id="indice"></a>
<h2><font color="#92002A" size=5>Índice</font></h2>

<br>

* [1. Introducción](#section1)
* [2. Acceso al área de trabajo de aprendizaje automático](#section2)
* [3. Obtención del modelo](#section2)
* [4. Implementación del script de entrada](#section3)
* [5. Creación del punto de conexión](#section4)

<br>

---

<br>


<a id="section1"></a>
## <font color="#92002A">1 - Introducción</font>
<br>

En esta libreta, vamos a hacer uso de los servicios de machine learning (*MLaaS*) que ofrece *Microsoft Azure* para crear un punto de conexión, desde el cual podremos utilizar el modelo entrenado para la predicción de playlist desde la *API REST* que desarrollemos más adelante.

Como vimos en la libreta anterior, creamos un área de trabajo de machine learning y una instancia de cómputo para entrenar nuestro modelo. únicamente nos faltaría registrar el modelo dentro del área de trabajo y montar el *endpoint* que hará uso de él. Esta tarea la realizaremos de forma programática desde la libreta.


<br>

<div class="alert alert-block alert-warning">
    
<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
    <strong>Importante</strong>: Aunque emplearemos las librerías de <i>Python</i> que ofrece <i>Microsoft</i> para hacer uso de los servicios de machine learning, es necesario que instalemos la <a href="https://docs.microsoft.com/es-es/cli/azure/install-azure-cli"><strong>Interfaz de la línea de comandos de Azure</strong></a> (también conocida como <i>Azure CLI</i>). Una vez instalada, debemos iniciar sesión en nuestra cuenta de Azure desde la línea de comandos. Una vez realizado este proceso, ya podremos utilizar sin problemas las librerías de <i>Azure ML</i> en <i>Python</i>.
</div>

<br>

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<br>

<a id="section2"></a>
## <font color="#92002A">2 - Acceso al área de trabajo de aprendizaje automático</font>

<br>

En esta sección, vamos a autenticarnos con nuestra cuenta de *Azure* para obtener acceso al área de trabajo de machine learning que tenemos creada. Podemos encontrar más información en la siguiente libreta de ejemplo: [Authentication in AzureML](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/authentication-in-azureml.ipynb)

In [1]:
#!pip install azureml azureml.core azureml-core

In [2]:
from azureml.core import Workspace
from azureml.core.authentication import AzureCliAuthentication

cli_auth = AzureCliAuthentication()

subscription_id = "<<INSERTAR IDENTIFICADOR>>" # Identificador de nuestra suscripción
resource_group = "<<INSERTAR NOMBRE DEL GRUPO DE RECURSOS>>" # Nombre del grupo de recursos donde esta el área de trabajo
workspace_name = "<<INSERTAR NOMBRE DEL ÁREA DE TRABAJO>>" # Nombre del área de trabajo

ws = Workspace(subscription_id, resource_group, workspace_name, auth=cli_auth)
ws.write_config()

print("Encontrado el área de trabajo '{}' en la región '{}'.".format(ws.name, ws.location))

Encontrado el área de trabajo 'serendipity-ml-workspace' en la región 'northeurope'.


<br>

A continuación. procedemos a registrar el modelo en la colección de modelos del área de trabajo de machine learning. Para identificarlo, le estableceremos un nombre, el framework del modelo y su correspondiente versión:

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<br>

<a id="section3"></a>
## <font color="#92002A">3 - Obtención del modelo</font>

<br>

Cuando realizamos el entrenamiento del modelo *LightFM* en la instancia de cómputo, empleando el script definido en la libreta *[Parte 3 - Modelo de recomendación](./03-ModeloRecomendacion.ipynb)*, incorporamos un fragmento de código para registrar dicho modelo en la biblioteca de modelos de nuestra área de trabajo. A continuación, vamos a importar el modelo que hemos empleado para crear el _endpoint_. En caso de que no se encontrara dicho modelo, por cualquier motivo, accederemos a la carpeta `model` de nuestra máquina y lo importaremos nuevamente:

In [3]:
from azureml.core.model import Model

model_name = 'serendipity-recsys-model'
model_version = 1

if model_name not in [m.name for m in  Model.list(workspace=ws)]:
    print("Modelo no encontrado. Registrando ...")
    model = Model.register(
        workspace = ws,
        model_path ='./model',
        model_name = model_name,
        model_framework="LightFM",
        model_framework_version="1.16.0",
        tags = {"version": model_version},
        description = "Serenditipy LightFM model for playlists recommendation (tracks)"
    )
    print("Modelo registrado")
else:
    for model in Model.list(workspace=ws):
        if model.name == model_name:
            model_version = str(model.version)
    print(f"Encontrado modelo '{model_name}', versión {model_version}")

Encontrado modelo 'serendipity-recsys-model', versión 1


<br>

<div class="alert alert-info">

<i class="fa fa-info-circle" aria-hidden="true"></i>
__Nota__: Debido al tamaño de nuestro modelo, en caso de necesitar registrarlo nuevamente, el proceso puede demorarse dependiendo de nuestra conexión a Internet.
</div>

<br>

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<br>

<a id="section4"></a>
## <font color="#92002A">4. Implementación del script de entrada</font>

<br>

El *script* de entrada se encarga de recibir los datos enviados y pasárselos al modelo para realizar la predicción. Seguidamente, devuelve la respuesta del modelo al cliente. El *script* es específico para dicho modelo y debe entender los datos que el modelo espera y devuelve.

Estas son las dos tareas que debe realizar en el script de entrada:

* Cargar el modelo (mediante una función llamada `init`).
* Ejecuta el modelo sobre los datos de entrada (mediante una función llamada `run`).

A continuación, se muestra el *script* de entrada que hemos desarrollado para nuestro servicio:

In [4]:
import os

SCRIPT_FOLDER = 'scripts/endpoint'

if not os.path.exists(SCRIPT_FOLDER):
    os.makedirs(SCRIPT_FOLDER)

In [5]:
%%writefile scripts/endpoint/recommend.py
import joblib
import json
import lightfm
import numpy as np
import os
import time

from scipy import sparse as sp

def init():
    global model
    global model_data
    global num_users
    global num_items
    global num_threads
    
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv("AZUREML_MODEL_DIR"), "model/model.pkl")
    model_data_path = os.path.join(os.getenv("AZUREML_MODEL_DIR"), "model/model_data.pkl")
    
    model = joblib.load(model_path)
    model_data = joblib.load(model_data_path)
    
    num_users = len([x for x in model_data['playlist_features_names'] if "name:" in x])
    num_items = len([x for x in model_data['track_features_names'] if "track:" in x])
    
    num_threads = 3
    

# Obtiene de un modelo los items similares al indicado
# mediante la similaridad coseno
def similar_items(item_id, model, num_items, N=1000):
    """
    :param item_id: Item del que obtener similares.
    :param model: Modelo a emplear.
    :param N: (Opcional) Número de items similares, por defecto se devuelven todos.
    :return: Lista de tuplas en formato (PID,SCORE).
    """
    
    if N > 1000:
        N = 1000
    N += 200
    (item_biased, item_representations) = model.get_item_representations()
    
    # Cosine Similarity
    scores = item_representations.dot(item_representations[item_id])
    item_norms = np.linalg.norm(item_representations, axis=1)
    item_norms[item_norms == 0] = 1e-10    
    scores /= item_norms
    best = np.argpartition(scores, -N)[-N:]    
    result = sorted(zip(best, scores[best] / item_norms[item_id]),
                  key=lambda x: -x[1])
    N -= 200
    
    return [{'item' : int(t), 'score' : float(s)} for (t,s) in result if t < num_items and t != item_id][:N]


# Obtiene de un modelo los usuarios similares al indicado
# mediante la similaridad coseno
def similar_users(user_id, model, num_users, N=1000):
    """
    :param item_id: Usuario del que obtener similares.
    :param model: Modelo a emplear.
    :param num_items: Número total de usuarios
    :param N: (Opcional) Número de usuarios similares, por defecto se devuelven todos.
    :return: Lista de tuplas en formato (PID,SCORE).
    """
    
    if N > 1000:
        N = 1000
    N += 200
    
    (user_biased, user_representations) = model.get_user_representations()
    
    # Cosine Similarity
    scores = user_representations.dot(user_representations[user_id])
    user_norms = np.linalg.norm(user_representations, axis=1)
    user_norms[user_norms == 0] = 1e-10    
    scores /= user_norms
    best = np.argpartition(scores, -N)[-N:] 
    
    result = sorted(zip(best, scores[best] / user_norms[user_id]),
                  key=lambda x: -x[1])
    N -= 200
    
    return [{'user' : int(u), 'score' : float(s)} for (u,s) in result if u < num_users and u != user_id][:N]


def get_similar_user_tags(tag_id, model, N=100):
    # Define similarity as the cosine of the angle
    # between the tag latent vectors

    # Normalize the vectors to unit length
    tag_embeddings = (model.user_embeddings.T
                      / np.linalg.norm(model.user_embeddings, axis=1)).T

    query_embedding = tag_embeddings[tag_id]
    similarity = np.dot(tag_embeddings, query_embedding)
    most_similar = np.argsort(-similarity)[1:N+1]
    
    result = []
    for user_id in most_similar:
        result.append({'user_tag_id' : int(user_id), 'score' : float(similarity[user_id])})

    return result


def get_similar_item_tags(tag_id, model, N=100):
    # Define similarity as the cosine of the angle
    # between the tag latent vectors

    # Normalize the vectors to unit length
    tag_embeddings = (model.item_embeddings.T
                      / np.linalg.norm(model.item_embeddings, axis=1)).T

    query_embedding = tag_embeddings[tag_id]
    similarity = np.dot(tag_embeddings, query_embedding)
    most_similar = np.argsort(-similarity)[1:N+1]
    
    result = []
    for item_id in most_similar:
        result.append({'item_tag_id' : int(item_id), 'score' : float(similarity[item_id])})

    return result


def format_userfeats(userfeat_ids, num_user_features):
    normalised_val = 1.0 
    new_user_features = np.zeros(num_user_features)
    
    for i in userfeat_ids:
        new_user_features[i] = normalised_val
    
    new_user_features = sp.csr_matrix(new_user_features)
      
    return new_user_features


def make_newuser_recomm(userfeat_ids, model, i_features, num_items, n_recs=1000, n_threads=1):
    if n_recs > 1000:
        n_recs = 1000
        
    total_userfeats = len(model.get_user_representations()[0])
    new_user_features = format_userfeats(userfeat_ids, total_userfeats)
    
    scores = model.predict(0, np.arange(num_items), user_features=new_user_features, 
                           item_features=model_data['track_features'], num_threads=n_threads)
    items = list(np.argsort(scores)[::-1])[:n_recs]
    
    result = []    
    for item_id in items:
        result.append({'item_id' : int(item_id), 'score' : float(scores[item_id])})
        
    return result


def make_user_recomm(user_id, model, u_features, i_features, num_items, n_recs=1000, n_threads=1):
    if n_recs > 1000:
        n_recs = 1000
        
    scores = model.predict(user_id, np.arange(num_items), user_features=u_features, 
                           item_features=i_features, num_threads=n_threads)
    items = list(np.argsort(scores)[::-1])[:n_recs]
    
    result = []    
    for item_id in items:
        result.append({'item_id' : int(item_id), 'score' : float(scores[item_id])})
        
    return result



def run(raw_data):
    input_json = json.loads(raw_data)
    
    if all(k in input_json.keys() for k in ("data","action")):    
        data = input_json["data"]
        action = input_json["action"]
        start_time = time.time()
        response = {}    
        
        try:
            if action == 'similar_items':
                data = json.loads(raw_data)["data"]
                result = similar_items(data,model,num_items)
                response['result'] = result 
            elif action == 'similar_users':
                data = json.loads(raw_data)["data"]
                result = similar_users(data,model,num_users)
                response['result'] = result
            elif action == 'similar_item_tags':
                data = json.loads(raw_data)["data"]
                result = get_similar_item_tags(data,model)
                response['result'] = result
            elif action == 'similar_user_tags':
                data = json.loads(raw_data)["data"]
                result = get_similar_user_tags(data,model)
                response['result'] = result
            elif action == 'make_prediction_newuser':
                data = json.loads(raw_data)["data"]
                result = make_newuser_recomm(data, model, model_data['track_features'], 
                                             num_items, n_threads=num_threads)
                response['result'] = result
            elif action == 'make_prediction_user':
                data = json.loads(raw_data)["data"]
                result = make_user_recomm(data, model, model_data['playlist_features'], 
                                          model_data['track_features'], num_items,
                                          n_threads=num_threads)
                response['result'] = result
            else:
                response['error'] = "Wrong invocation."
            
        except Exception as e:
            response['error'] = str(e)
            
        response['execution_time'] = time.time() - start_time

        return response
    else:
        return {'error' : 'Wrong invocation.'}

Overwriting scripts/endpoint/recommend.py


<br>

El script `recomend.py`, puede realizar las siguientes tareas:

1) Encontrar pistas similares a una dada.
2) Encontrar playlists similares a una dada.
3) Buscar características de pistas similares a la indicada.
4) Buscar características de playlists similares a la indicada.
5) Realizar predicciones sobre una playlist conocida (está en nuestro conjunto de datos).
6) Realizar predicciones a una nueva playlist (no se dispone información de ella).

<br>

Con el *script* de entrada ya definido, continuamos con la creación del punto de conexión.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<br>

<a id="section5"></a>
## <font color="#92002A">5. Creación del punto de conexión</font>

<br>

Para crear el punto de conexión, primero vamos a definir un *AciWebservice*. Este elemento representa un modelo de aprendizaje automático implementado como punto de conexión de servicio web en *Azure Container Instances*.

In [6]:
from azureml.core.webservice import AciWebservice

# create deployment config i.e. compute resources
aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=2,
    memory_gb=12,
    tags={"data": "MPD", "method": "lightfm"},
    description="Recommend playlists with LightFM",
)

<br>

Seguidamente, vamos a definir un *environment* de *Python* con los paquetes necesarios para la ejecución del *script* y lo guardaremos en un fichero *YML*:

In [7]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.environment import Environment

serenditipyenv = CondaDependencies()
serenditipyenv.set_python_version("3.8.10")
serenditipyenv.add_pip_package("lightfm==1.16.0")
serenditipyenv.add_pip_package("numpy")
serenditipyenv.add_pip_package("scipy")

ENV_FILE_PATH = os.path.join(SCRIPT_FOLDER, 'serenditipyenv.yml')
with open(ENV_FILE_PATH,"w") as f: 
    f.write(serenditipyenv.serialize_to_string())

# Create environment
env = Environment.from_conda_specification(
    name="serenditipyenv", 
    file_path=ENV_FILE_PATH)

<br>

Por último, definimos la configuración de inferencia y desplegamos el servicio (punto de conexión):

In [8]:
%%time
import uuid
from azureml.core.model import InferenceConfig
from azureml.core.model import Model

model = Model(workspace=ws, name=model_name, version=model_version)

# create an inference config i.e. the scoring script and environment
inference_config = InferenceConfig(
    entry_script="recommend.py", 
    source_directory=SCRIPT_FOLDER,
    environment=env)

# deploy the service
service_name = "serendipity-recsys-model-" + str(uuid.uuid4())[:4]
service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[model],
    inference_config=inference_config,
    deployment_config=aciconfig,
)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-09-01 15:39:58+02:00 Creating Container Registry if not exists.
2021-09-01 15:39:58+02:00 Registering the environment.
2021-09-01 15:40:00+02:00 Use the existing image.
2021-09-01 15:40:00+02:00 Generating deployment configuration.
2021-09-01 15:40:01+02:00 Submitting deployment to compute.
2021-09-01 15:40:06+02:00 Checking the status of deployment serendipity-recsys-model-fab6..
2021-09-01 15:46:40+02:00 Checking the status of inference endpoint serendipity-recsys-model-fab6.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Wall time: 7min 29s


<br>

Una vez que está disponible nuestro punto de conexión al modelo que hemos creado, vamos a probar que funciona correctamente. Para ello, vamos a realizar una predicción a la playlist `1000002`:

In [18]:
import json
import requests

input_data = {
    "action" : "make_prediction_user",
    "data" : 1000002
}

input_data = json.dumps(input_data)
headers = {"Content-Type": "application/json"}

resp = requests.post(service.scoring_uri, input_data, headers=headers)
resp_json = json.loads(resp.text)

In [19]:
resp_json['execution_time']

2.2925477027893066

In [20]:
resp_json['result'][:10]

[{'item_id': 5411, 'score': -110.66651916503906},
 {'item_id': 14272, 'score': -110.7023696899414},
 {'item_id': 5313, 'score': -110.80838012695312},
 {'item_id': 1330, 'score': -110.93283081054688},
 {'item_id': 6164, 'score': -111.12303161621094},
 {'item_id': 6044, 'score': -111.17489624023438},
 {'item_id': 20870, 'score': -111.26298522949219},
 {'item_id': 3764, 'score': -111.34845733642578},
 {'item_id': 5397, 'score': -111.37860870361328},
 {'item_id': 3806, 'score': -111.46448516845703}]

<br>

En caso de querer eliminar el punto de conexión que acabamos de crear, para evitar gastos innecesarios, basta con ejecutar la siguiente celda:

In [21]:
# if you want to keep workspace and only delete endpoint (it will incur cost while running)
service.delete()

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-graduation-cap" aria-hidden="true" style="color:#92002A"></i> </font></div>